In [1]:
def Velu(P, Q):
    assert (P.order().is_prime())
    E = P.curve()
    return E.isogeny(P)(Q)

In [2]:
l = 2 # fix the prime l

In [35]:
# aux(n, P) = f_n(P)
def aux(n, P):
    assert (P.order() == power(l, n)) # redundant step, but useful when debugging
    if n == 0:
        return []
    else:
        if n == 1:
            return [P]
        else:
            m = n // 2
            R = power(l, n - m) * P # R = l^{n-m} P is of order l^m
            # print ("R is of order = l^", log(R.order(), l))
            f = aux(m, R)
            phiRP = compute(m, R, P, f)
            # print ("phiRP is of order = l^", log(phiRP.order(), l))
            return f + aux(n - m, phiRP) # the concatenation of lists is given by `+`


# compute(n, P, Q, fnP) = image of Q in E/<P>, given fnP = f_n(P) = aux(n, P)
def compute(n, P, Q, fnP = None):
    # print("This is the", cnt_compute, "call to compute, n =", n, ", P is of order =l^", log(P.order(), l))
    assert (P.order() == power(l, n)) # redundant step, but useful when debugging
    imQ = Q # image of Q under each step
    if fnP == None:
        fnP = aux(n, P)
    for i in range(n): # permute from i = 0 to i = n - 1
        imQ = Velu(fnP[i], imQ)
        if i != n - 1: # not necessary, but could be useful if we implement aux() differently
            if (imQ.curve() != fnP[i+1].curve()) :
                isom = imQ.curve().isomorphism_to(fnP[i+1].curve())
                imQ = isom(imQ)
    return imQ


In [26]:
# Generate a sample to test
p_test = 89
F_test = GF(power(p_test, 2))
E = EllipticCurve(F_test, [-1, 0])
E.abelian_group()

Additive abelian group isomorphic to Z/200 + Z/40 embedded in Abelian group of points on Elliptic Curve defined by y^2 = x^3 + 88*x over Finite Field in z2 of size 89^2

In [15]:
def get_point_order_power_l (E, n, N): # A naive way to find a point of order l^n on E
    for i in range(N):
        P = E.random_point()
        if P.order() == l ^ n:
            return P
    return "Failed"

In [32]:
P_test = get_point_order_power_l(E, 3, 10000)
print(P_test, P_test.order())

(54*z2 + 40 : 26*z2 + 21 : 1) 8


In [33]:
for i in range(4):
    Q_test = E.random_point()
    print(compute(3, P_test, Q_test) == E.isogeny(P_test)(Q_test))
    # I am surprised that this is an equality (rather than isomorphism), and that I need to compute no isomorphism in `compute`.

True
True
True
True


In [36]:
# Question 2.8
p = 2 ^ 127 - 1
Fq.<a> = GF(power(p, 2))
E = EllipticCurve(Fq, [-1, 0])
[P, Q] = E.gens()

In [37]:
# verify that both P and Q have order 2 ^ 127
P.order() == 2 ^ 127 & Q.order() == 2 ^ 127

True

In [38]:
# compute R = \varphi_P(Q)
R = compute(127, P, Q)
R

(159641116237164011577138424333518592728*a + 50151477678775928310586280658106159686 : 97212599674500559127419380079867975655*a + 52160980509111977692718439083884729272 : 1)

In [39]:
R == E.isogeny(P)(Q)

True